In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import sys
import boto3
from boto3 import session
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import zipfile
import glob

In [2]:
#Checking older versions
#locOld = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/austin-2010-2018-central/2018/" 
#mode_choice_raw = pd.read_csv(locOld + 'final_trip_mode_choice_raw.csv')
#utilities = pd.read_csv(locOld + 'final_trip_mode_choice_utilities.csv')

#### Reading the files

In [3]:
loc = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2010-lessrh-20220223/activitysim/" 
trips = pd.read_csv(loc + 'final_trips.csv')
households = pd.read_csv(loc + 'final_households.csv')
persons = pd.read_csv(loc + 'final_persons.csv')
tours = pd.read_csv(loc +'final_tours.csv')
plans = pd.read_csv(loc +'final_plans.csv')
landuse = pd.read_csv(loc +'final_land_use.csv')
#checkpoints = pd.read_csv(loc + 'final_checkpoints.csv')
#joint_tour_participants = pd.read_csv(loc + 'final_joint_tour_participants.csv')
#specs = pd.read_csv(loc +'final_trip_mode_choice_specs.csv')
#tripModeChoice = pd.read_csv(loc +'final_trip_mode_choice_raw.csv')
#tripModeChoiceUtilities = pd.read_csv(loc +'final_trip_mode_choice_utilities.csv')

In [4]:
#Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

#### Merging the files

In [5]:
#Merge households and persons 
persons = persons.sort_values(by=['household_id'])
households = households.sort_values(by=['household_id'])
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [6]:
#Merge tours, households and persons
tours = tours.sort_values(by=['person_id'])
hhpersons = hhpersons.sort_values(by=['person_id'])
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [7]:
%%time
#Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id'])
hhperTours = hhperTours.sort_values(by=['person_id','tour_id'])
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

Wall time: 40.5 s


#### Don't forget to merge plans files here

In [ ]:
%%time   #### Check this code
#Merge trips, tours, households and persons, and plans
plans = plans.sort_values(by=['person_id', 'trip_id'])
tourTripsMerged = tourTripsMerged.sort_values(by=['person_id','trip_id'])
plansTripsMerged = pd.merge(left=plans, right=tourTripsMerged, how='left', on=['person_id','trip_id'], suffixes=('', '_drop'))
plansTripsMerged.drop([col for col in plansTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [20]:
tours[tours['person_id'] == 4]

,tour_id,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,origin,household_id,tdd,start,end,duration,composition,destination_logsum,tour_mode,mode_choice_logsum,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose
469973,189,4,othdiscr,1,1,1,1,non_mandatory,1,927.0,1005.0,865,146.0,15.0,16.0,1.0,NaN,13.156275,DRIVEALONEFREE,-2.239582,NaN,NaN,0out_0in,othdiscr


In [17]:
#Merge trips, tours, households, and persons with land-use
landuse = landuse.sort_values(by=['TAZ'])
tourTripsMerged = tourTripsMerged.sort_values(by=['TAZ'])
landuseTripsMerged = pd.merge(tourTripsMerged, landuse, how='left', left_on='TAZ', right_on='TAZ', suffixes=('', '_drop'))
landuseTripsMerged.drop([col for col in landuseTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [34]:
#Concat mode_choice_raw files
path = "C:/Users/nazanin/Downloads/trip_mode_choice/trip_mode_choice/"
all_files = glob.glob(path + "*raw.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_raw = pd.concat(li2, axis = 0, ignore_index=True)

In [35]:
#Concat mode_choice_utilities files
path = "C:/Users/nazanin/Downloads/trip_mode_choice/trip_mode_choice/"
all_files = glob.glob(path + "*utilities.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_utilities = pd.concat(li2, axis = 0, ignore_index=True)

In [42]:
#Merge mode_choice_raw and utilities
SFmode_choice_raw = SFmode_choice_raw.sort_values(by=['trip_id'])
SFmode_choice_utilities = SFmode_choice_utilities.sort_values(by=['trip_id'])
rawUtil = pd.merge(left=SFmode_choice_raw, right=SFmode_choice_utilities, how='left', on='trip_id', suffixes=('', '_drop'))
rawUtil.drop([col for col in rawUtil.columns if 'drop' in col], axis=1, inplace=True)

In [44]:
%%time
#Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
landuseTripsMerged = landuseTripsMerged.sort_values(by=['trip_id'])
rawUtil = rawUtil.sort_values(by=['trip_id'])
SFActMerged= pd.merge(left=landuseTripsMerged, right=rawUtil, how='left', on=['trip_id'], suffixes=('', '_drop'))
SFActMerged.drop([col for col in SFActMerged.columns if 'drop' in col], axis=1, inplace=True)

Wall time: 1min 40s


In [48]:
SFActMerged.shape

(1856570, 613)

In [50]:
SFActMerged_chunk = SFActMerged[0:1000]
SFActMerged_chunk.to_csv('C:/Shared-Work/Data/CleanData/SFActMerged_chunk.csv', index = False)

In [ ]:
specs.Expression_id = specs.Expression_id .astype(str)
specs.rename(columns = {'Expression_id':'trip_mode'}, inplace = True)
specs = specs.set_index('trip_mode')

In [ ]:
specsT = specs.transpose()

In [ ]:
ActSimSpec = pd.merge(ActSimMerged, specsT, left_on = 'trip_mode', right_index=True, how = 'left')

In [ ]:
#ActSimSpec.to_csv('C:/Shared-Work/Data/CleanData/Final_ActSimMerged.csv')